# The interesting topics in this year

In [1]:
import pandas as pd

def filter_by_topic(df, topic):
    df['google_scholar_link'] = df['title'].apply(lambda x: 'https://scholar.google.com/scholar?q=' + x.replace(' ', '+'))
    df = df.drop_duplicates(subset=['title'])
    return df[df['topic'].str.contains(topic, case=False, na=False)].copy()

In [2]:
from semanticscholar import SemanticScholar
import time
from tqdm import tqdm
tqdm.pandas()

def get_paper_citation(title):
    sch = SemanticScholar()
    time.sleep(1)
    title = title.replace('-', ' ')
    try:
        papers = sch.search_paper(title)
        if len(papers) > 0:
            return papers[0].citationCount
    except:
        print(f'Error in get citation for {title}')
    
    return 0
    

def get_topic_citations(df):
    df['ss_citations'] = df['title'].progress_apply(get_paper_citation)
    return df

In [3]:
def get_conf_topics(conf_path='data/2023/5_iclr.csv_topics.csv', topic='multimodal'):
    df_conf = pd.read_csv(conf_path)
    df_conf_topic = filter_by_topic(df_conf, topic=topic)
    print(len(df_conf_topic), len(df_conf))
    df_conf_topic = get_topic_citations(df_conf_topic)
    return df_conf_topic[['title','year','source','ss_citations','google_scholar_link']]

In [4]:
def get_conf_topics_all(conf_path='data/2023/5_iclr.csv_topics.csv', topics=['multimodal']):
    df_conf = pd.read_csv(conf_path)

    save_path = conf_path + '_emerging.xlsx'
    writer = pd.ExcelWriter(save_path, engine="xlsxwriter")

    for topic in topics:
        print(f"topic {topic}")
        df_conf_topic = get_conf_topics(conf_path, topic)
        df_conf_topic.to_excel(writer, sheet_name=topic, index=False)
    writer.close()
    print(f'Saved to {save_path}')

## 2023, ICLR, May

In [6]:
# get_conf_topics_all(conf_path='data/2023/5_iclr.csv_topics.csv', topics=['multimodal', 'diffusion models', 'llm', 'self-supervision'])

## 2023, CVPR, June

In [7]:
# get_conf_topics_all(conf_path='data/2023/6_cvpr.csv_topics.csv', topics=['3d', 'zero_few-shot', 'transformer', 'segmentation', 'multimodal', 
# 'imitation learning', 'radiance field', 'motion', 'diffusion models', 'multi-agent'])

## 2023, ICML, July

In [5]:
# get_conf_topics_all(conf_path='data/2023/7_icml.csv_topics.csv', topics=['zero_few-shot', 'graph', 'optimization', 'reinforcement learning', 'representation',
# 'motion', 'decision trees', 'radiance field', 'diffusion models', 'federated learning'])

## 2023, ICCV, Oct.

In [6]:
get_conf_topics_all(conf_path='data/2023/10_ICCV.csv_topics.csv', topics=['object detection', '3d', 'segmentation', 'transformer', 'zero_few-shot'])

topic object detection
2156 2156


  0%|          | 4/2156 [00:15<3:05:09,  5.16s/it]

Error in get citation for Diff Retinex: Rethinking Low light Image Enhancement with A Generative Diffusion Model


  0%|          | 7/2156 [00:20<1:40:37,  2.81s/it]

Error in get citation for A Dynamic Dual Processing Object Detection Framework Inspired by the Brain's Recognition Mechanism


...

Error in get citation for SSF: Accelerating Training of Spiking Neural Networks with Stabilized Spiking Flow


 96%|█████████▌| 178/185 [05:48<00:14,  2.05s/it]

Error in get citation for CroCo v2: Improved Cross view Completion Pre training for Stereo Matching and Optical Flow


100%|██████████| 185/185 [06:03<00:00,  1.96s/it]


Saved to data/2023/10_ICCV.csv_topics.csv_emerging.xlsx


## 2023, NIPS, Dec.

In [7]:
get_conf_topics_all(conf_path='data/2023/12_nips.csv_topics.csv', topics=['object detection', 'zero_few-shot', 'graph', 'optimization', 'llm', 'radiance field', 'motion', 'differential equation solving', 'rl'])

topic object detection
3218 3218


  0%|          | 2/3218 [00:01<48:46,  1.10it/s]

Error in get citation for Information Maximization Perspective of Orthogonal Matching Pursuit with Applications to Explainable AI


  1%|          | 35/3218 [01:17<2:18:03,  2.60s/it]

Error in get citation for Locality Sensitive Hashing in Fourier Frequency Domain For Soft Set Containment Search


...

Error in get citation for Survival Permanental Processes for Survival Analysis with Time Varying Covariates


 86%|████████▌ | 106/123 [04:10<00:33,  1.96s/it]

Error in get citation for History Filtering in Imperfect Information Games: Algorithms and Complexity


100%|██████████| 123/123 [04:46<00:00,  2.33s/it]


Saved to data/2023/12_nips.csv_topics.csv_emerging.xlsx
